In [2]:
import csv
import itertools
import numpy
import random
from CWHashingFamily import CWHashingFamily

SHINGLES_LEN = 4
PRIME = 997
BINS = 900

def shingles(text, shingleSize):
    return [text[i:i+shingleSize] for i in xrange(0,len(text)-(shingleSize+1))]


def minhash(shinglesList, seed):
    random.seed(seed)
    minhash = -1
    for shingle in shinglesList:
        cwFactory = CWHashingFamily(BINS, PRIME)
        numericHash = cwFactory.numericFunction(random.randrange(1,PRIME), random.randrange(0,PRIME))
        stringHash = cwFactory.stringFunction(random.randrange(0,PRIME),numericHash)
        shHash = stringHash.hash(shingle)
        if ((minhash == -1) or (shHash < minhash)):
            minhash = shHash
    return minhash

In [2]:
ID = 0
TEXT = 9
R = 4
B = 8

trainRDD = sc.textFile("train_sin_repeticiones.csv").map(lambda line: line.split(","))

trainRDD = trainRDD.filter(lambda line: line[ID] != "Id")

trainRDD = trainRDD.map(lambda line: (line[ID], shingles(line[TEXT],SHINGLES_LEN)))

trainRDD = trainRDD.map(lambda line: (line[0], [ minhash(line[1], i) for i in xrange(0, R*B) ] )).take(5)

print trainRDD


[(u'492346', [3, 2, 0, 9, 1, 0, 1, 3, 6, 0, 0, 5, 14, 9, 0, 0, 0, 0, 7, 1, 3, 0, 1, 0, 2, 0, 0, 0, 3, 2, 0, 3]), (u'343811', [15, 5, 6, 3, 1, 3, 3, 0, 4, 0, 1, 2, 1, 0, 4, 6, 0, 4, 4, 1, 0, 5, 1, 1, 5, 5, 0, 3, 0, 5, 1, 2]), (u'347257', [20, 1, 3, 19, 24, 11, 8, 0, 33, 10, 2, 3, 21, 19, 10, 8, 2, 10, 12, 24, 0, 5, 24, 25, 1, 5, 6, 28, 0, 1, 3, 9]), (u'225529', [25, 29, 67, 38, 64, 6, 0, 42, 85, 39, 37, 2, 14, 18, 2, 73, 82, 2, 0, 125, 240, 21, 39, 3, 23, 2, 17, 68, 32, 29, 45, 20]), (u'485488', [1, 1, 12, 0, 0, 7, 3, 0, 5, 0, 1, 4, 0, 0, 5, 0, 0, 5, 5, 0, 1, 3, 0, 2, 1, 3, 5, 5, 0, 1, 3, 3])]


In [3]:
from LSH import LSH
from Record import Record

lsh = LSH(R,B)

for i in xrange(len(trainRDD)):
    record = Record((trainRDD[i][0], trainRDD[i][1]))
    lsh.add(record)


Adding record:492346 to LSH at POS:57
table 0 :[u'492346']
Adding record:492346 to LSH at POS:68
table 1 :[u'492346']
Adding record:492346 to LSH at POS:7
table 2 :[u'492346']
Adding record:492346 to LSH at POS:87
table 3 :[u'492346']
Adding record:492346 to LSH at POS:42
table 4 :[u'492346']
Adding record:492346 to LSH at POS:38
table 5 :[u'492346']
Adding record:492346 to LSH at POS:31
table 6 :[u'492346']
Adding record:492346 to LSH at POS:53
table 7 :[u'492346']
Adding record:343811 to LSH at POS:46
table 0 :[u'343811']
Adding record:343811 to LSH at POS:40
table 1 :[u'343811']
Adding record:343811 to LSH at POS:50
table 2 :[u'343811']
Adding record:343811 to LSH at POS:51
table 3 :[u'343811']
Adding record:343811 to LSH at POS:93
table 4 :[u'343811']
Adding record:343811 to LSH at POS:89
table 5 :[u'343811']
Adding record:343811 to LSH at POS:21
table 6 :[u'343811']
Adding record:343811 to LSH at POS:56
table 7 :[u'343811']
Adding record:347257 to LSH at POS:44
table 0 :[u'347257'

In [4]:
text = "i got them in a very timely manner and they're all very large and green. there seems to be an inordinate amount of the seeds e out a weird beige color, but it still tastes great and is a much better deal than any of the indian markets in town. cheers to frontier."
test = (492346,shingles(text,4))
test = (test[0], [ minhash(test[1], i) for i in xrange(0, R*B)] )
testRecord = Record(test)

In [5]:
print lsh.getAllSimilarRecords(testRecord)

obtained pos:26 for record (<bound method Record.id of <Record.Record instance at 0x7f5e883ff170>>,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 0
obtained pos:49 for record (<bound method Record.id of <Record.Record instance at 0x7f5e883ff170>>,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 1
obtained pos:10 for record (<bound method Record.id of <Record.Record instance at 0x7f5e883ff170>>,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 2
obtained pos:60 for record (<bound method Record.id of <Record.Record instance at 0x7f5e883ff170>>,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])at table 3
obtained pos:16 for record (<bound method Record.id of <Record.Record instance at 0x7f5e883ff170>>,[1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 

(492346, [1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])
(492346, [1, 0, 1, 0, 1, 1, 2, 1, 0, 5, 2, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 3, 0, 0, 0, 0, 1, 0, 2, 2])
